In [ ]:
from google.colab import drive
drive.mount('/content/drive')

def inception_base(version, base_folder="/content/drive/MyDrive/",epoch=30):
  epoch=epoch
  base_folder=base_folder
  gen_folder = 'VERSION'+str(version)
  print(gen_folder)
  import numpy as np
  import pandas as pd
  from sklearn import preprocessing
  from sklearn.model_selection import train_test_split
  import tensorflow as tf
  import tensorflow.keras as keras
  import time
  from sklearn.metrics import accuracy_score

  # common_name = "class_code_202006_202206_all_classes_down_25k"
  data_folder = base_folder + gen_folder +"/"+"down/"
  output_directory = base_folder + gen_folder +"/"+"LSTM_models/"
  saved_RMS = np.load(data_folder + "scaled.npy") # data path has to be replaced with shared file
  saved_RMS_label = np.load(data_folder + "label.npy")

  label_1 = []
  file_1 = []
  pos_codes = []
  phase_codes = []
  ground_codes = []
  waveform_ids = []
  phase_ground_code = []          
  for i in saved_RMS_label:
    label_1.append(int(i[0]))
    pos_codes.append(int(i[1]))
    phase_codes.append(int(i[2]))
    ground_codes.append(int(i[3]))
    file_1.append(i[4])
    waveform_ids.append(i[5])
    #[AN BN CN AB BC CA ABC ABN BCN CAN ABCN UNK]
    if phase_codes[-1] > 2 and phase_codes[-1] < 7 and ground_codes[-1] == 0:
      phase_ground_code.append(phase_codes[-1]  + 4)
    elif  phase_codes[-1] < 7:
      phase_ground_code.append(phase_codes[-1])
    else:
      phase_ground_code.append(6)
      
  label_df = pd.DataFrame(
      {'label': label_1,
      'file_name': file_1,
      'pos_codes':pos_codes,
      'phase_codes':phase_codes,
      'ground_codes':ground_codes,
      'waveform_ids':waveform_ids,
      'phase_ground_code': phase_ground_code
      })
  labels_unique = label_df.label.unique()
  print(labels_unique)

  lb = preprocessing.LabelBinarizer()
  lb.fit(labels_unique)
  lb.classes_

  ind_nan = (np.unique((np.nonzero(np.isnan(saved_RMS))[0])))
  ## For class
  Y = lb.transform(label_df.label) #lb.transform(saved_RMS_label) #fit_transform(saved_RMS_label)
  X = saved_RMS
  X[ind_nan] = saved_RMS[0,:,:]
  Y[ind_nan] = Y[0]
  del saved_RMS
  del saved_RMS_label
  print('Deleting saved_RMS and Label')

  x_train,x_val, y_train, y_val = train_test_split(X,Y,test_size=0.25,random_state=10)

  del X
  del Y
  print('Local instance X and Y')

  nb_classes = len(lb.classes_) #17 # Number of classes
  input_shape=(120,40) # Input shape

  model_name ="InceptionTime_down_base_"+gen_folder
  class Classifier_INCEPTION:

      def __init__(self, output_directory, input_shape, nb_classes,  build=True, batch_size=64, lr=0.001,
                  nb_filters=24, use_residual=True,verbose = True, use_bottleneck=True, depth=16, kernel_size=30, nb_epochs=epoch):

          self.output_directory = output_directory

          self.nb_filters = nb_filters
          self.use_residual = use_residual
          self.use_bottleneck = use_bottleneck
          self.depth = depth
          self.kernel_size = kernel_size - 1
          self.callbacks = None
          self.batch_size = batch_size
          self.bottleneck_size = 24
          self.nb_epochs = nb_epochs
          self.lr = lr
          self.verbose = verbose
          self.model_name = model_name

          if build == True:
              self.model = self.build_model(input_shape, nb_classes)
              if (verbose == True):
                  self.model.summary()
              self.model.save_weights(self.output_directory.replace('MODEL_NAME',self.model_name ) + self.model_name + '.h5') # for every run you need to change the path

      def _inception_module(self, input_tensor, stride=1, activation='linear'):

          if self.use_bottleneck and int(input_tensor.shape[-1]) > self.bottleneck_size:
              input_inception = keras.layers.Conv1D(filters=self.bottleneck_size, kernel_size=1,
                                                    padding='same', activation=activation, use_bias=False)(input_tensor)
          else:
              input_inception = input_tensor

          # kernel_size_s = [3, 5, 8, 11, 17]
          kernel_size_s = [self.kernel_size // (2 ** i) for i in range(3)]

          conv_list = []

          for i in range(len(kernel_size_s)):
              conv_list.append(keras.layers.Conv1D(filters=self.nb_filters, kernel_size=kernel_size_s[i],
                                                  strides=stride, padding='same', activation=activation, use_bias=False)(
                  input_inception))

          max_pool_1 = keras.layers.MaxPool1D(pool_size=3, strides=stride, padding='same')(input_tensor)

          conv_6 = keras.layers.Conv1D(filters=self.nb_filters, kernel_size=1,
                                      padding='same', activation=activation, use_bias=False)(max_pool_1)

          conv_list.append(conv_6)

          x = keras.layers.Concatenate(axis=2)(conv_list)
          x = keras.layers.BatchNormalization()(x)
          x = keras.layers.Activation(activation='relu')(x)
          return x

      def _shortcut_layer(self, input_tensor, out_tensor):
          shortcut_y = keras.layers.Conv1D(filters=int(out_tensor.shape[-1]), kernel_size=1,
                                          padding='same', use_bias=False)(input_tensor)
          shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

          x = keras.layers.Add()([shortcut_y, out_tensor])
          x = keras.layers.Activation('relu')(x)
          return x

      def build_model(self, input_shape, nb_classes):
          input_layer = keras.layers.Input(input_shape)

          x = input_layer
          input_res = input_layer

          for d in range(self.depth):

              x = self._inception_module(x)

              if self.use_residual and d % 3 == 2:
                  x = self._shortcut_layer(input_res, x)
                  input_res = x

          gap_layer = keras.layers.GlobalAveragePooling1D()(x)

          output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

          model = keras.models.Model(inputs=input_layer, outputs=output_layer)

          model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(self.lr),
                        metrics=['accuracy'])

          reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5,min_lr=0.0001)
          es = keras.callbacks.EarlyStopping( monitor="val_loss", patience=10,
                                      verbose=1,  restore_best_weights=True)
          #file_path = self.output_directory + 'best_model_inception_ce67k_v1.h5' # for every run you need to change the path
          file_path = (self.output_directory.replace('MODEL_NAME',self.model_name ) + 'best_' + self.model_name + '.h5') 

          model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='val_loss',mode='auto',save_best_only=True)

          self.callbacks = [reduce_lr, model_checkpoint, es]

          return model

      def fit(self, x_train, y_train, x_val, y_val):
          if not tf.test.is_gpu_available:
              print('error no gpu')
              exit()
          # x_val and y_val are only used to monitor the test loss and NOT for training

          if self.batch_size is None:
              mini_batch_size = int(min(x_train.shape[0] / 10, 16))
          else:
              mini_batch_size = self.batch_size

          start_time = time.time()

          hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=self.nb_epochs,
                                verbose=self.verbose, validation_data=(x_val, y_val), callbacks=self.callbacks)

          duration = time.time() - start_time
          file_path = (self.output_directory.replace('MODEL_NAME',self.model_name ) + 'last_' + self.model_name + '.h5') 
          self.model.save(file_path) # for every run you need to change the path
          keras.backend.clear_session()
          return duration

      def predict(self, x_test,):
          start_time = time.time()
          model_path = (self.output_directory.replace('MODEL_NAME',self.model_name ) + 'best_' + self.model_name + '.h5') 
          model = keras.models.load_model(model_path)
          y_pred = model.predict(x_test, batch_size=16)
          y_pred = np.argsort(y_pred, axis=1)
          return y_pred
      


  cf = Classifier_INCEPTION(output_directory,input_shape,nb_classes)
  cf.fit( x_train = x_train, y_train=y_train, x_val = x_val, y_val= y_val )
  model= cf
  pred = cf.predict(x_val)
  list_pred= []
  for i in pred:
    list_pred.append(i[-1])
  list_test = []
  for i in y_val:
    list_test.append(np.where(i == 1)[0][0])
  accuracy= accuracy_score(list_pred, list_test)
  print('Accuracy : '+str(accuracy))
  return model,accuracy



if __name__== '__main__':
  model,accuracy = inception_base(version=5,epoch=50)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
VERSION5
[10110 10120 12110 13170 15110 30200 20210 20520 20510 20140 20130 23100
 23190 13120 13121 23310 13110]
Deleting saved_RMS and Label
Local instance X and Y
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 120, 40)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 120, 24)      960         ['input_1[0][0]']                
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 120, 40)      0           ['input_1[0][0]']              